## CH3 평가
### 다양한 평가기준들
- 평가 관련 용어
	- 정분류
		- True-Positive: 실제 결과값이 1 인것을 양성(1)으로 예측한다 
		- True- Negatvie: 실제 결과값이 0 인것을 음성(0)으로 예측한다
	- 오분류
		- False- Positive: 양성(1)이라 예측했지만 실제 결과값이 0이다
		-  False- Negative: 음성(0)이라 예측했지만 실제 결과값이 1이다
- 다양한 평가기준들
	- 정확도 $accuracy=\frac{TP+TN}{TP+TN+FP+FN}$ : 모든 예측중 제대로 예측한 것의 비율
	- 정밀도 $precision=\frac{TP}{TP+FP}$ : 참이라고 예측한 것중 실제로 참인 것의 비율
	- 재현율/민감도 $recall/sensitivity=\frac{TP}{TP+FN}$ :실제로 참인것들 중 참이라고 예측한 비율
		- FN가 발생해선 안되는 업무에서 중요지표가 된다. 
		- 예) 암검진
	- 특이도 $specificity=\frac{TN}{TN+FP}$ : 실제로 거짓인 것중 거짓이라 예측한 것의 비율
	- FP가 발생해선 안되는 업무에서 중요지표가 된다
	- 예) 업무메일을 스팸으로 차단하는 경우

### 다양한 평가기준을 사용하는 이유 - 자동검진 기계와 타이타닉
- 높은 정확도면 좋은 알고리즘인가?: 1000명중 1명이 걸리는 희귀병이 있다하자. 이 병을 자동으로 진단해주는 기계가 있는데 아무도 모르게 어느날 고장이 나서 무조건 음성판정을 내린다고 하자.  이 기계의 정확도는 999/1000이지만 아무짝에도 쓸데가 없다. 왜냐면 *예측한 것이 들어맞는 비율이 높은것 보다도, 실제 양성인 환자가 양성으로 판별되는 비율이 중요하기 때문이다.*
- 타이타닉에서 여성이면 생존, 남성이면 사망이라고만 분류하는 단순한 이 알고리즘도 0.788의 높은 정확도를 가지고 있다
```python
#BaseEstimator: Base class for all estimators in scikit-learn.  
class MyDummyClassifier(BaseEstimator):  
    #fit 메소드는 아무것도 학습시키지 않는다  
    def fit(self,x,y=None):  
        pass  
    # predict() 메소드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측한다  
    def predict(self,X):  
        pred= np.zeros((X.shape[0],1))  
        for i in range(X.shape[0]):  
            if X['Sex'].iloc[i] == 1:  
                pred[i] = 0  
            else:  
                pred[i] = 1  
        return pred  
  
#챕터 2에서 만들어둔 모듈을 가져옴  
from titanic_preprocessing import preprocessing_feature  
#원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터를 분할한다  
titanic_df =pd.read_csv('./train.csv')  
y_titanic_df= titanic_df['Survived']  
X_titanic_df = titanic_df.drop('Survived',axis=1)  
X_titanic_df= preprocessing_feature(X_titanic_df)  
  
X_train, X_test, y_train, y_test= train_test_split(X_titanic_df,y_titanic_df,test_size=0.2,random_state=0)  
  
#위의 Dummy_Classifier 모델을 이용하여 학습/예측 평가해보자  
myclf= MyDummyClassifier()  
myclf.fit(X_train,y_train)  
  
mypredictions= myclf.predict(X_test)  
print(f'Dummy Classifier의 정확도는 {accuracy_score(y_test,mypredictions):.3f}')

____________________________________________________________
result)
Dummy Classifier의 정확도는 0.788
```

## 2. 오차행렬 / 혼돈행렬 confounding matrix
- 입력
	- `sklearn.metrics.confusion_matrix(y_true y_pred)`
- 출력
	- 다음의 값을 만족하는 $C_{ij}$ 로 구성된 행렬이 만들어진다
		- $C_{ij}$ 는 $j$ 번 클래스로 예측되고, $i$번이 실제의 값인 데이터의 수다.
		- 즉 $j$ 번째 열은 $j$ 번째 클래스를 예측하는 것이고, $i$ 번째 행은 $i$ 번째 실제 클래스인것이다

## 3. 정밀도와 재현율의 Tradeoff
- 정밀도와 재현율의 *tradeoff*
	- 정밀도 $precision=\frac{TP}{TP+FP}$ 와 재현율 $recall=\frac{TP}{TP+FN}$ 은 상호보완적인 평가지표로서 한쪽을 높이면 다른 한쪽의 수치가 떨어지기 쉽다
-  `predict_proba`
	- 다음의 값을 만족시키는 $P_{ij}$로 구성된 행렬이 만들어진다
	- $P_{ij}$ 는 $j$번 클래스로 예측되고, $i$번째 값이 되는 함수이다
	- `pred` 메서드는 `predict_proba` 에 기반한 메서드로서, 대부분의 경우 `predict_proba` 의 `i` 번째 행에서 가장 큰 확률값을 갖는 $P_{i\tilde{j}}$ 를 찾아, $\tilde{j}$를 클래스 값으로 예측한다
	- 그러나 더 일반적으로는 각 클래스의 확률이 특정 기준을 넘어서면 해당 클래스로 예측하는데, 이때 기준값을 임계값threshold라고 한다.
	
-  임계값을 설정하여 예측하기
```python
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix  
  
def get_clf_eval(y_test,pred):  
    confusion= confusion_matrix(y_test,pred)  
    accuracy = accuracy_score(y_test,pred)  
    precision= precision_score(y_test,pred)  
    recall= recall_score(y_test,pred)  
    print('오차 행렬')  
    print(confusion)  
    print(f'정확도 {accuracy:.3f} 정밀도 {precision:.3f} 재현율 {recall:.3f} ')  
  
#Binzarizer의 threshold 설정값. 분류 결정 임계값임  
custom_threshold= 0.5  
#predit_proba()의 반환값의 두번째 컬럼, 즉 positive 클래스 컬럼중 하나만 추출하여 Binarizer를 적용시킨다  
pred_proba_1 = pred_proba[:,1].reshape(-1,1)  
  
binarizer= Binarizer(threshold=custom_threshold).fit(pred_proba_1)  
custom_predict= binarizer.transform(pred_proba_1)  
get_clf_eval(y_test,custom_predict)

-----------------------------------------------------------------------
result)
오차 행렬
[[108  10]
 [ 16  45]]
정확도 0.855 정밀도 0.818 재현율 0.738
```
- 임계값을 0.5에서 0.4로 수정한 경우
```python
#Binarizer의 threshold 설정값을 0.4로 설정. 즉 분류 결정 임계값을 0.5에서 0.4로 낮춘다  
custom_threshold= 0.4  
pred_proba_1= pred_proba[:,1].reshape(-1,1)  
binarizer= Binarizer(threshold=custom_threshold).fit(pred_proba_1)  
custom_predict= binarizer.transform(pred_proba_1)  
  
get_clf_eval(y_test,custom_predict)
________________________________________________________________________
result)
오차 행렬
[[98 20]
 [11 50]]
정확도 0.827 정밀도 0.714 재현율 0.820
```
- 위의 두 예제에서 보듯 임계값을 낮추니 재현율이 올라가고 정밀도가 떨어지게 되었다. 왜 그럴까?
- 증명
	- 임계값을 낮출수록 Positive 예측수가 늘어난다. 이를 $PP$라고 표기하자
	- $PP=TP+FP$ 이다
	- 재현율 $reacall=\frac{TP}{TP+FN}$ 이고, 정밀도 $precision=\frac{TP}{TP+FP}$ 라는 식을 고려하자
	- 재현율을 보자 이경우  $PP$ 가 증가한다 해서 $FN$ 가 증가하지 않으므로 $TP$ 가 유일하게 변하는 변수이다 
		- $\frac{\partial{Recall}}{\partial{TP}}=\frac{1}{TP+FN}+TP\cdot\frac{\partial{}}{\partial{TP}}\cdot\frac{1}{(TP+FN)}=\frac{1}{TP+FN}-\frac{TP}{(TP+FN)^2}=\frac{FN}{(TP+FN)^2}>0$
	- 정밀도를 보자. $PP$ 가 증가하는 경우 , 
		- $\frac{\partial{Precision}}{\partial{PP}}=\frac{-TP}{PP^2}++\frac{1}{PP}\cdot\frac{\partial{TP}}{\partial{PP}}=\frac{1}{PP}(\frac{\partial{TP}}{\partial{PP}}-\frac{TP}{PP})$
		- $FP$ 만 증가하는 경우 
			- 재현율 $recall$ 가 증가하진 않는다
			- 정밀도 $\frac{\partial{Precision}}{\partial{FP}}=-\frac{TP}{(TP+FP)^2}$ 로 정밀도가 떨어진다
	- 종합적으로 보면 
		- 임계값을 낮추면 재현율은 항상 높아진다
		- 임계값을 낮추었을 때 $TP$의 비율이 높은경우 정밀도가 높아지고, $FP$의 비율이 높은경우 정밀도가 낮아진다 ( 그러나 그래프에서 보면 대체로 항상 낮아지는 듯)
		- ![](Pasted%20image%2020220720113044.png)
	- 정밀도가 100%가 되기 위해선 확실히 기준이 되는 경우에만 positive로 예측하고 나머지는 모두 negative로 예측한다
	- 재현율이 100%가 되기 위해선 모든 분류를 positive로 예측한다

### 4.F-1 스코어
- 정밀도와 재현율을 결합한 지표이다
- 정밀도와 재현율이 어느 한쪽으로 치우치지 않을 때 상대적으로 높은 값을 갖는다
- $F_1=\frac{2}{\frac{1}{Recall}+\frac{1}{Precision}}=2\cdot \frac{Precision \cdot Recall}{Precision+Recall}$
- $dF_1=\frac{\partial F_1}{\partial(Recall)}\cdot d(Recall)+\frac{\partial F_1}{\partial{Precision}}\cdot d(Precision)$
- $=2(-\frac{Precision \cdot Recall}{(Precision+Recall)^2}+\frac{Precision}{Precision+Recall})\cdot d(Recall)+2(-\frac{Precision \cdot Recall}{(Precision+Recall)^2}+\frac{Recall}{Precision+Recall})\cdot d(Precision)$
- $=2(\frac{Precision^2}{(Precision+Recall)^2}d(Recall)+\frac{Recall^2}{(Precision+Recall)^2}d(Precision))$ ( 증명 미완..)

## 5.ROC와 관련 측정 개념들
- $FPR- False \,Postive\, Rate=\frac{FP}{TN+FP}$: 실제 음성중에서 잘못 양성으로 예측하는 비율 
- $TNR-True\,Negate\,Rate$-특이성 $\,Specificity= \frac{TN}{TN+FP}$ : 실제 음성중 음성이라 예측하는 비율
- $ROC=\frac{TPR}{FPR}=\frac{Recall}{FPR}$
- 그래프 시각화하기
```python
def roc_curve_plot(y_test,pred_proba_c1):
fprs, tprs, thresholds= roc_curve(y_test,pred_proba_c1)
#ROC 곡선을 그래프 곡선으로 그린다
plt.plot(fprs,tprs,label='ROC')
#가운데 대각선 직선을 그린다
plt.plot([0,1],[0,1],'k--',label='random')
# FPR X 축의 scale을 0.1 단위로 변경. X축 Y축명 설정등
start, end = plt.xlim()
plt.xticks(np.round(np.arange(start,end,0.1),2))
plt.xlim(0,1); plt.ylim(0,1)
plt.xlabel('FPR(1-Sensitivity'); plt.ylabel('TPR(Recall)')
plt.legend()
  

roc_curve_plot(y_test,pred_proba[:,1])
```



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5a548db0-9bec-4ed6-b090-9c062f32efdb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>